yfinance and keras tuner installed as an extra library

In [ ]:
#IMPORT LIBRARIES
import math
import pandas_datareader as pdr
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
import matplotlib.pyplot as plt
import yfinance as yfin
import datetime
import keras_tuner as kt
import tensorflow as tf
import tensorflow_addons as tfa

yfin.pdr_override()
plt.style.use('fivethirtyeight')


Using TensorFlow backend


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Pull data into data frame
Then print it out/display it

In [ ]:
df = yfin.download("BTC-USD", start="2020-07-02", end="2021-08-03")
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-07-02,9231.139648,9274.962891,9036.623047,9123.410156,9123.410156,16338916796
2020-07-03,9124.842773,9202.344727,9058.794922,9087.303711,9087.303711,13078970999
2020-07-04,9084.233398,9183.295898,9053.629883,9132.488281,9132.488281,12290528515
2020-07-05,9126.090820,9162.183594,8977.015625,9073.942383,9073.942383,12903406143
2020-07-06,9072.849609,9375.474609,9058.664062,9375.474609,9375.474609,17889263252
...,...,...,...,...,...,...
2021-07-29,39995.453125,40593.070312,39352.058594,40008.421875,40008.421875,27167146027
2021-07-30,40027.484375,42235.546875,38397.355469,42235.546875,42235.546875,33072782960
2021-07-31,42196.304688,42231.449219,41110.832031,41626.195312,41626.195312,25802845343


In [ ]:
# Mount Google Drive to get Bitcoin News Titles csv file
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get Titles

titles = pd.read_csv('/content/drive/MyDrive/bitcoin_titles.csv')
titles = titles[['Date', 'Title']]
titles.head()

FileNotFoundError: ignored

In [ ]:
# Group Titles by Date
tl1 = titles.groupby([titles['Date']])['Title'].apply('ş0 '.join).reset_index()
tl1

In [ ]:
# Separate titles at put them in to the same row by date
for i in range(len(tl1['Date'])):
  temp = np.array(tl1['Title'][i].split('ş0 '))
  test = pd.DataFrame(temp)
  tl1 = tl1.append(test.T, ignore_index=True)
  tl1['Date'] = tl1.Date.shift(1)

cols = tl1.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]
tl1 = tl1[cols]

tl1.drop(titles.index[0:398],0,inplace=True)
tl1

In [ ]:
# Drop the first row and reset index to start from 0
tl1.drop('Title',axis=1, inplace=True)
tl1.reset_index(drop=True, inplace=True)
tl1

In [ ]:
# Merge the two dataframes
tl1['Date'] = tl1['Date'].astype('datetime64')
df = pd.merge(df, tl1, on='Date', how='outer') # here Date is common column

df

In [ ]:
from ta.volatility import BollingerBands
indicator_bb = BollingerBands(close=df["Close"], window=20, window_dev=2)

# Add Bollinger Bands features
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

# Add Bollinger Band high indicator
df['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

# Add Bollinger Band low indicator
df['bb_bbli'] = indicator_bb.bollinger_lband_indicator()

# Add Width Size Bollinger Bands
df['bb_bbw'] = indicator_bb.bollinger_wband()

# Add Percentage Bollinger Bands
df['bb_bbp'] = indicator_bb.bollinger_pband()

In [ ]:
# Plot the data
plt.figure(figsize=(16,8))
plt.title('Close price histroy')
plt.plot(df['Close'],color='white')
plt.plot(df['bb_bbh'],color='darkorange')
plt.plot(df['bb_bbl'],color='darkblue')
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close price usd $', fontsize=18)
plt.show()

In [ ]:
# Option 1: Create a new dataframe with only the 'Close' column
#data1= df['bb_bbh']
#data2= df['bb_bbl']
#data3= data1-data2
#data = pd.DataFrame()
#data['Close'] = pd.Series(data3)


# Option 2: Create a new dataframe using Bollinger Bands features
data = df[['Close','bb_bbm','bb_bbh','bb_bbl','bb_bbli','bb_bbhi','bb_bbw','bb_bbp','Open','High','Low','Adj Close','Volume']]
dataset = data.values #convert the dataframe to a numpy array

#Get the number of rows to train the model on
training_data_len = math.ceil( len(dataset) * .75)
training_data_len#print
data

In [ ]:
# Scale the data to be values between 0 and 1
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Create the training data set
train_data = scaled_data[0:training_data_len , :]
x_train = []
y_train = []

# Use 60 days of data to predict the 61st day
for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])

In [ ]:
# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1)) # 3D
x_train

In [ ]:
# Create the testing data set
test_data = scaled_data[training_data_len-60: , :]
x_test = []
y_test = []
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])
  y_test.append(test_data[i, 0])

In [ ]:
# Convert the data to a numpy array
x_test, y_test = np.array(x_test),np.array(y_test)

In [ ]:
# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_test

In [ ]:
# Control the shapes of the data
print(x_test.shape)
print(x_train.shape)

print(y_test.shape)
print(y_train.shape)

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import GridSearchCV

In [ ]:
from tensorflow.keras import activations,layers
# Write a builder function that takes hyperparameters as input and returns a compiled Keras model
def model_builder(hp):
    # Create a Sequential model with the tunable hyperparameters
    model = Sequential()
    # Add the LSTM layers and Dropout Regularization
    model.add(LSTM(hp.Int('input_unit',min_value=32,max_value=128,step=32), return_sequences=True, input_shape= ( x_train.shape[1],1)))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)))
    model.add(LSTM(hp.Int('input_unit2',min_value=32,max_value=512,step=32), return_sequences=True, input_shape= ( x_train.shape[1],1)))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)))
    model.add(LSTM(hp.Int('input_unit3',min_value=32,max_value=384,step=32), return_sequences=True, input_shape= ( x_train.shape[1],1)))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)))

    # RMSprop optimizer can be used for RNNs
    #model.add(RMSprop(hp.Int(learning_rate=0.001,rho=0.9,momentum=0.0,epsilon=1e-07,centered=False)))

    # Add multiple LSTM layers with tunable hyperparameters
    for i in range(hp.Int('n_layers', 1, 3)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units0',min_value=32,max_value=384,step=32),return_sequences=True))
        model.add(Dropout(hp.Float(f'Dropout_{i}_rate',min_value=0,max_value=0.3,step=0.1)))

    # Add the LSTM layer with tunable hyperparameters 
    model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=384,step=32)))
    model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.3,step=0.1)))
    model.add(Dense(30, activation=hp.Choice('dense_activation',values=['sigmoid'])))
    model.add(Dense(1, activation=hp.Choice('dense_activation',values=['relu'])))

    # Compile the model with Adam optimizer and Mean Squared Error loss
    model.compile(loss='mean_absolute_error', optimizer='adam',metrics = ['mse'])

    return model

In [ ]:
# Alternative way to define the model using the ARIMA model

#from statsmodels.tsa.arima_model import ARIMA
#def model_builder(hp):
    #model = Sequential()
    #model.add(ARIMA(train_data,order=(0,1,2)))
    #model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)))
    #return model

#tuner = kt.RandomSearch(model_builder, objective="mse", max_trials = 1, executions_per_trial = 1,directory = "./")
    ##model.add(ARIMA(hp.Int('input_unit2',min_value=32,max_value=512,step=32), input_shape= ( x_train.shape[1],1)))
    ##model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)))
    ##model.add(ARIMA(hp.Int('input_unit3',min_value=32,max_value=384,step=32), input_shape= ( x_train.shape[1],1)))
    ##model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)))

In [ ]:
# Initialize the Keras Tuner RandomSearch object with the model_builder function
tuner = kt.RandomSearch(model_builder, objective="mse", max_trials = 1, executions_per_trial = 1,directory = "./")

# Bayesian Optimization Tuner can also be used
#tuner = kt.BayesianOptimization(max_trials=10,num_initial_points=2,alpha=0.0001,beta=2.6,hypermodel = Tuner.run_trial(x_train))

# Initialize the TimeStopping callback to stop training when the given time is reached
time_stopping_callback = tfa.callbacks.TimeStopping(seconds= 20000, verbose=1)

# Initialize the EarlyStopping callback to stop training when the model is not improving
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

# Search for the best hyperparameters
tuner.search(x=x_train, y=y_train, epochs = 200, batch_size = 64, validation_split=0.1, callbacks=[stop_early, tf.keras.callbacks.TensorBoard("/tmp/tb_logs"), time_stopping_callback])

In [ ]:
from plot_keras_history import show_history, plot_history
# Show the best hyperparameters
model = tuner.get_best_models(num_models=1)[0]
model.summary()
model.fit(x_train,y_train,epochs=200,batch_size=64,validation_data=(x_test, y_test))

# Show the training history
#val_loss = model.history['val_loss']
#loss = model.history['loss']
#history=model.fit(x_train,y_train,epochs=50,batch_size=64,validation_data=(x_test, y_test))
#val_loss = history.history['val_loss']
#loss = history.history['loss']
#plt.plot(loss)
#plt.plot(val_loss)
#plt.legend(['loss', 'val_loss'])
#plt.show()
model.save("model.h5")

In [ ]:
from tensorflow import keras
# Load the model
model = keras.models.load_model('model.h5')

In [ ]:
# Get the predicted values
predictions = model.predict(x_test)

# Unscale the values using the MinMaxScaler
#predictions = scaler.inverse_transform(predictions)[0]
predictions = scaler.fit_transform(predictions)

%load_ext tensorboard

%tensorboard --logdir /tmp/tb_logs

In [ ]:
# Calculate the root mean squared error
rmse = np.sqrt( np.mean(predictions - y_test)**2)
rmse

In [ ]:
# Normalize the RMSE

#normalized = rmse/(max(dataset)-min(dataset))
#print(min(dataset))
#print(max(dataset))
#normalized

In [ ]:
# Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions.T[0]

plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Actual', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# Show the valid and predicted prices
valid

In [ ]:
valid.keys()

In [ ]:
valid.reset_index(inplace=True, drop=True)
valid

In [ ]:
valid['Close'][0]

In [ ]:
valid['Predictions'][0]

In [ ]:
# Calculate the number of correct and incorrect predictions
correct = 0
incorrect = 0
for i in range(len(valid)-1):
  if valid['Predictions'][i] < valid['Predictions'][i+1]:
    if valid['Close'][i] < valid['Close'][i+1]:
      correct = correct + 1
    else:
      incorrect = incorrect + 1
  elif valid['Predictions'][i] > valid['Predictions'][i+1]:
    if valid['Close'][i] > valid['Close'][i+1]:
      correct = correct + 1
    else:
      incorrect = incorrect + 1


print("----")
print('Correct predictions: .3f' correct)
print('Incorrect predictions: .3f' incorrect)
print("----")
# Calculate the accuracy of the model
acc = correct/(correct+incorrect)
print('Accuracy of the model: %.3f %%' %((100*acc)))
